In [1]:
import os, arcpy
import pandas as pd

## Get workspace
workspace = arcpy.env.workspace

## Check for the number of fires mapped and compare to expected
## Check that the burn severity matches perimeter area (may be slightly diffrent)
fc = "interim_burn_severity_2025"

unique_values = set(row[0] for row in arcpy.da.SearchCursor(fc, "FIRE_NUMBER"))
firenums = list(unique_values)
print('Unique Fire Numbers:',len(firenums))

## Export as dataframe
columns = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] #List the fields you want to include. I want all columns except the geometry
df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, columns), columns=columns)
print('Burn Severity Area (ha):',sum(df['AREA_HA']))

Unique Fire Numbers: 1
Burn Severity Area (ha): 10098.992981815403


In [2]:
## Check Geometry
arcpy.management.CheckGeometry(
    in_features=fc,
    out_table=os.path.join(workspace,"geom_check_ogc"),
    validation_method="OGC"
)

<Result 'C:\\ArcPro_Projects\\BurnSevMaps\\BurnSevMaps.gdb\\geom_check_ogc'>

In [3]:
## If needed run Repair Geometry
arcpy.management.RepairGeometry(
    in_features=fc,
    delete_null="DELETE_NULL",
    validation_method="OGC"
)

<Result 'interim_burn_severity_2025'>

In [4]:
#Check for all fields we need to have 
fields = ['FIRE_NUMBER','FIRE_YEAR','BURN_SEVERITY_RATING',
         'PRE_FIRE_IMAGE','PRE_FIRE_IMAGE_DATE',
         'POST_FIRE_IMAGE','POST_FIRE_IMAGE_DATE',
         'AREA_HA','COMMENTS']

for f in fields:
    if f in columns:
        print(f,"exists")
    else:
        print(f,"f doesn't exist!")

FIRE_NUMBER exists
FIRE_YEAR exists
BURN_SEVERITY_RATING exists
PRE_FIRE_IMAGE exists
PRE_FIRE_IMAGE_DATE exists
POST_FIRE_IMAGE exists
POST_FIRE_IMAGE_DATE exists
AREA_HA exists
COMMENTS exists


In [5]:
#Print fields that aren't in the list
#OBJECTID, FEATURE_AREA_SQM, FEATURE_LENGTH_M, Shape_Length, Shape_Area are okay.
for c in columns:
    if c not in fields:
        print(c)


OBJECTID
FEATURE_AREA_SQM
FEATURE_LENGTH_M
Shape_Length
Shape_Area


In [6]:
## Check fire year and burn severity rating
print(df['FIRE_YEAR'].unique())
print(df['BURN_SEVERITY_RATING'].unique())

['2025']
['Medium' 'High' 'Unburned' 'Low' 'Unknown']


In [7]:
## Check for nulls in PRE_FIRE_IMAGE and POST_FIRE_IMAGE
s1 = df[df['PRE_FIRE_IMAGE'].isnull()]
print('PRE_FIRE_IMAGE is null:', s1['FIRE_NUMBER'].unique())

s2 = df[df['POST_FIRE_IMAGE'].isnull()]
print('POST_FIRE_IMAGE is null:', s2['FIRE_NUMBER'].unique())

s3 = df[df['PRE_FIRE_IMAGE_DATE'].isnull()]
print('PRE_FIRE_IMAGE_date is null:', s3['FIRE_NUMBER'].unique())

s4 = df[df['POST_FIRE_IMAGE_DATE'].isnull()]
print('POST_FIRE_IMAGE_DATE is null:', s4['FIRE_NUMBER'].unique())

PRE_FIRE_IMAGE is null: []
POST_FIRE_IMAGE is null: []
PRE_FIRE_IMAGE_date is null: []
POST_FIRE_IMAGE_DATE is null: []


In [8]:
## Check for unique COMMENTS
print(df['COMMENTS'].unique())

[None]
